In [1]:
import requests
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
from personal import *

url = "http://127.0.0.1:5000"
# url = "https://turnitin-api.herokuapp.com"


s = requests.Session()

In [2]:
login_result = s.post(url + "/login", json={
    "email": USERNAME,
    "password": PASSWORD
})
auth = login_result.json()

classes_result = s.post(url + "/classes", json=auth)
classes = classes_result.json()
for c in classes:
    if "World Lit" in c["title"]:
        world_lit = c
print(world_lit)

first_class_data = dict(auth, url=world_lit["url"])
assignments_result = s.post(url + "/assignments", json=first_class_data)
assignments = assignments_result.json()
assignments[-5]

{'title': 'World Literature: Period 5', 'url': 'https://www.turnitin.com/s_class_portfolio.asp?r=86.4369854138154&svr=23&lang=en_us&aid=5245&cid=22703844'}


{'aid': '93021882',
 'dates': {'due': '06/30/2020 23:59:00',
  'post': '06/12/2020 00:00:00',
  'start': '06/04/2020 15:34:00'},
 'file': 'void',
 'info': 'No additional instructions have been entered for this assignment. Late paper submissions are allowed for this assignment. Resubmissions are not allowed for this assignment. Similarity reports are available immediately after submission.',
 'oid': 'void',
 'submission': 't_submit.asp?r=97.7174878803524&svr=25&lang=en_us&aid=93021882',
 'title': "Revision 1: Kento's Test Assignment (for Kento only)"}

In [3]:
len(assignments)

29

In [4]:
assignments[-6]

{'aid': '93021867',
 'dates': {'due': '06/30/2020 23:59:00',
  'post': '06/12/2020 00:00:00',
  'start': '06/04/2020 15:33:00'},
 'file': 'void',
 'info': 'No additional instructions have been entered for this assignment. Late paper submissions are allowed for this assignment. Resubmissions are not allowed for this assignment. Similarity reports are available immediately after submission.',
 'oid': 'void',
 'submission': 't_submit.asp?r=97.7174878803524&svr=25&lang=en_us&aid=93021867',
 'title': "Kento's Test Assignment (for Kento only)"}

In [5]:
ass = assignments[-6]
aid = ass["aid"]
cookies = auth["auth"]
author_first = "Kento"
author_last = "Bento"
with open("../SampleSubmission.docx", 'rb') as fin:
    filebytes = fin.read()
submission_title = "Custom turnitin API"

In [6]:
submit_url = "https://www.turnitin.com/" + ass["submission"]
query = {"aid": aid, "session-id": cookies["session-id"], "lang": "en_us"}
form_data = dict(
    async_request=1,
    author_first=author_first, 
    author_last=author_last,
    title=submission_title,
    filename="SampleSubmission.docx",
    referer=submit_url,
    userfile=filebytes,
    db_doc='',
    dropbox_filename='',
    google_doc='',
    google_auth_uri='',
    token='',
    submit_button=''
)

alternate_fd = [
    {"name": k, "value": v}
    for k, v in form_data.items()
]

In [7]:
[{i: form_data[i]} for i in form_data if i != "userfile"]

[{'async_request': 1},
 {'author_first': 'Kento'},
 {'author_last': 'Bento'},
 {'title': 'Custom turnitin API'},
 {'filename': 'SampleSubmission.docx'},
 {'referer': 'https://www.turnitin.com/t_submit.asp?r=97.7174878803524&svr=25&lang=en_us&aid=93021867'},
 {'db_doc': ''},
 {'dropbox_filename': ''},
 {'google_doc': ''},
 {'google_auth_uri': ''},
 {'token': ''},
 {'submit_button': ''}]

In [9]:
ass["submission"]

't_submit.asp?r=97.7174878803524&svr=25&lang=en_us&aid=93021867'

In [10]:
r = s.post(submit_url, data=form_data, headers={"accept": "application/json"},params=query, cookies=cookies)

In [11]:
display(HTML(r.text))

In [12]:
rr = s.post("https://www.turnitin.com/panda/get_submission_metadata.asp", params={
    "session-id": cookies["session-id"],
    "lang": "en_us",
    "skip_ready_check": 0,
    # "uuid": NotImplemented
})

In [13]:
confirmation_url = "https://www.turnitin.com/submit_confirm.asp?"
c_query = {
    "lang": "en_us",
    "session-id": cookies["session-id"],
    "data-state": "confirm"
}
r = s.post(confirmation_url, params=query, cookies=cookies)